In [1]:
import numpy as np
from tqdm import tqdm
from math import floor,ceil,sqrt

In [2]:
def arbre_recomb(n,b):
    l=2*b*n+1
    S=np.zeros((l,n+1))
    for i in range(l):
        S[i,n]= b*n-i
    for j in range(1,n+1):
        for i in  range(l):
            if i>=b*j and i<=b*(2*n-j):
                S[i,n-j]=b*n-i
    return S

In [3]:
def proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n):
    proba=np.zeros((3,len(List_sigma)))
    h=T/n
    for i in range(len(List_sigma)):
        proba[0,i]=(List_sigma[i]**2+List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[1,i]=(List_sigma[i]**2-List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[2,i]=1-(proba[0,i]+proba[1,i])
    return proba

In [4]:
def transition_matrix(Q,h):
    nbr=Q.shape[0]
    tr_matrix=np.zeros((nbr,nbr))
    for i in range(nbr):
        for j in range(nbr):
            if i==j:
                tr_matrix[i,j]=np.exp(Q[i,j]*h)
            else:
                tr_matrix[i,j]=-(1-np.exp(Q[i,i]*h))*Q[i,j]/Q[i,i]
    return tr_matrix         

In [55]:
# initialization for two regime model
T = 1.0
m =2
b = 2
n = 1000
step = 1/n
sigma_bar = 0.2
arbre=arbre_recomb(n,m)
Q=np.array([[-0.5,0.5],[0.5,-0.5]])
Pa=transition_matrix(Q,step)

sigmas = np.array([0.15,0.25])
li=np.array([1,2])
R=np.array([0.05,0.05])
a = R-0.5*sigmas**2


In [11]:
def prix_call_europeen(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)

    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
        
    for j in tqdm(range(1,n+1)):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [12]:
A=prix_call_europeen(100,96,n,b,sigma_bar,T,sigmas,a,li,R,Pa)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:19<00:00, 50.32it/s]


In [19]:
Pa[:,0]

array([9.99500125e-01, 4.99875021e-04])

In [58]:
def prix_call_europeen2(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    m = len(List_sigma)
    table_prix = np.zeros((l,m))

    # intialization of the prices at time n
    table = -K+S_o*np.exp(sigma_bar*np.sqrt(h)*arbre[:,n].reshape(l,1))
    table = np.maximum(table,0.0)
    table_prix += table
    print(table_prix)


    for t in range(1,n+1):
        for i in range(l-b*2*t):
            table = np.zeros(m)
            for reg in tqdm(range(m)):
                r = List_r[reg]
                saut = Liste_saut[reg]
                table[reg] = np.exp(-r*h)*np.dot(P[reg],np.sum(table_prix[[i+2-saut,i+2+saut,i+2]]*proba_umd[:,reg].reshape(-1,1),axis = 0))
            table_prix[i,:] = table


    return table_prix[0,:]

In [26]:
A[2000,0]

{0: 6.923020326921834, 1: 9.317382007445124}

In [27]:
np.array([A[i,900].values() for i in range(4001) if A[i,900]!=0])

array([dict_values([8440363.709401505, 8440363.746236034]),
       dict_values([8387149.986841507, 8387150.023443808]),
       dict_values([8334271.755379335, 8334271.791750874]), ...,
       dict_values([0.0, 0.0]), dict_values([0.0, 0.0]),
       dict_values([0.0, 0.0])], dtype=object)

In [28]:
A[:,1000]

array([{0: 29902584.95457939, 1: 29902584.95457939},
       {0: 29714060.56423974, 1: 29714060.56423974},
       {0: 29526724.744284555, 1: 29526724.744284555}, ..., {0: 0, 1: 0},
       {0: 0, 1: 0}, {0: 0, 1: 0}], dtype=object)

In [59]:
B=prix_call_europeen2(100,96,n,b,sigma_bar,T,sigmas,a,li,R,Pa)

[[29902584.95457945 29902584.95457945]
 [29714060.5642398  29714060.5642398 ]
 [29526724.74428461 29526724.74428461]
 ...
 [       0.                0.        ]
 [       0.                0.        ]
 [       0.                0.        ]]


100%|██████████| 2/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [57]:
B

array([6.92302033, 9.31738201])

In [ ]:
def prix_put_europeen(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [ ]:
def prix_call_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],-K+S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix

In [ ]:
def prix_put_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],K-S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix

In [52]:
# initialization for four regimes
sigmas = np.array([0.9,0.5,0.7,0.2])
R = np.array([0.02,0.1,0.06,0.15])
a = R-0.5*sigmas**2

T= 1
N = 1000
step = T/N
sigma_ = 0.4
b= 4
m = 4 # number of regimes

Q = np.ones((m,m))/3
for i in range(m):
    Q[i,i] = -1
print('Q\n',Q)

Pa = np.zeros((m,m))
for i in range(m):
    for j in range(m):
        if i!=j:
            Pa[i,j]= -(1-np.exp(Q[i,i]*step))*Q[i,j]/Q[i,i]
        else:
            Pa[i,i]= np.exp(Q[i,i]*step)
print('Transition matrix\n',Pa)

arbre=arbre_recomb(n,m)

K = 100
So = 120

# define the li so that the probabilities of up,middle, down are all positives
L = [0]*m
for i in range (m):
    k1,k2 = floor(2*sigmas[i]/sigma_),ceil(2*sigmas[i]/sigma_)
    if k1==k2 or k1*sigma_<sigmas[i]:
        L[i] = k2
    else:
        c = ((k1*sigma_)**2-sigmas[i]**2)/a[i]**2
        d = (k2*sigma_-sqrt((k2*sigma_)**2-4*sigmas[i]**2))**2/(4*a[i])**2
        if c<=d:
            L[i] = k2
        else:
            L[i] = k1
print("The increments Li", L)


Q
 [[-1.          0.33333333  0.33333333  0.33333333]
 [ 0.33333333 -1.          0.33333333  0.33333333]
 [ 0.33333333  0.33333333 -1.          0.33333333]
 [ 0.33333333  0.33333333  0.33333333 -1.        ]]
Transition matrix
 [[9.99000500e-01 3.33166722e-04 3.33166722e-04 3.33166722e-04]
 [3.33166722e-04 9.99000500e-01 3.33166722e-04 3.33166722e-04]
 [3.33166722e-04 3.33166722e-04 9.99000500e-01 3.33166722e-04]
 [3.33166722e-04 3.33166722e-04 3.33166722e-04 9.99000500e-01]]
The increments Li [4, 2, 3, 1]


In [43]:
A=prix_call_europeen(100,96,n,b,sigma_bar,T,sigmas,a,li,R,Pa)

100%|██████████| 1000/1000 [00:56<00:00, 17.75it/s]


In [54]:
B=prix_call_europeen2(100,96,n,b,sigma_bar,T,sigmas,a,L,R,Pa)

[[9.31427674e+12 9.31427674e+12 9.31427674e+12 9.31427674e+12]
 [9.25555398e+12 9.25555398e+12 9.25555398e+12 9.25555398e+12]
 [9.19720144e+12 9.19720144e+12 9.19720144e+12 9.19720144e+12]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]


 19%|█▊        | 187/1000 [00:55<04:01,  3.37it/s]


KeyboardInterrupt: 

In [2]:
b = 2
n = 2500
l = 2*b*n+1
test_A = np.random.normal(0,1,size= (l,10))
p_A = np.random.random((3,10))
sauts = np.random.randint(1,b+1,size = 10)
test_B = test_A.copy()
p_A

array([[0.45585886, 0.41741637, 0.30432589, 0.69977093, 0.57496197,
        0.58240799, 0.97472275, 0.32224489, 0.25722476, 0.19104263],
       [0.64376835, 0.95684128, 0.47750834, 0.19871936, 0.58250321,
        0.97213879, 0.17105622, 0.82869024, 0.66327585, 0.8358949 ],
       [0.04490112, 0.34844242, 0.95172453, 0.1677405 , 0.41329007,
        0.69884885, 0.3966882 , 0.77254428, 0.33807054, 0.49118655]])

In [5]:
for t in tqdm(range(1,n+1)):
    for i in range(20-2*b*t):
        table = np.zeros(10)
        for reg in range(10):
            saut = sauts[reg]
            for k in range(10):
                table[reg] += test_A[i+2-saut,k]*p_A[0,reg]+test_A[i+2,k]*p_A[1,reg]+test_A[i+2+saut,k]*p_A[2,reg]
        test_A[i] = table

100%|██████████| 2500/2500 [00:00<00:00, 213415.83it/s]


In [6]:
for t in tqdm(range(1,n+1)):
    for i in range(20-2*b*t):
        table = np.zeros(10)
        for reg in range(10):
            saut = sauts[reg]
            table[reg] = np.sum(test_B[[i+2-saut,i+2,i+2+saut]]*p_A[:,reg].reshape(-1,1))
        test_B[i] = table

100%|██████████| 2500/2500 [00:00<00:00, 152575.63it/s]


In [5]:
test_A[0]

array([[ 2.70346898e-01, -1.22529447e-01,  1.15531784e+00,
        -3.96124940e+00,  1.86925429e+00,  6.37123917e-01,
        -3.20958324e+00, -3.20685631e+00,  2.60919109e-02,
         1.62512439e+00],
       [-8.45027010e-01, -1.98227188e+00, -1.21232309e+00,
        -1.31866219e+00, -5.29127239e+00, -4.48015418e+00,
         5.46780151e-02,  7.30423600e-01, -1.39740386e+00,
        -1.86011809e+00],
       [-3.94458090e-01, -5.11301006e-01, -3.30410115e+00,
        -3.29332415e+00, -3.62216436e+00,  3.92814684e-02,
        -4.75009527e+00, -1.02597828e+00,  3.06765160e-01,
        -4.75611619e+00],
       [-1.52461762e+00, -5.06263414e-01, -4.89835164e-01,
        -5.44598729e+00, -4.25420002e+00, -3.18466352e+00,
        -3.70165001e+00, -4.37637666e+00, -3.38096409e-01,
         9.18341025e-01],
       [ 1.18024366e-01, -3.11018024e+00,  2.75901422e+00,
        -3.81183859e+00,  4.41686111e-01, -3.27361812e+00,
        -2.14783101e+00,  1.14980027e-01, -1.94375906e+00,
         3.

In [6]:
test_B[0]

array([[ 2.70346898e-01, -1.22529447e-01,  1.15531784e+00,
        -3.96124940e+00,  1.86925429e+00,  6.37123917e-01,
        -3.20958324e+00, -3.20685631e+00,  2.60919109e-02,
         1.62512439e+00],
       [-8.45027010e-01, -1.98227188e+00, -1.21232309e+00,
        -1.31866219e+00, -5.29127239e+00, -4.48015418e+00,
         5.46780151e-02,  7.30423600e-01, -1.39740386e+00,
        -1.86011809e+00],
       [-3.94458090e-01, -5.11301006e-01, -3.30410115e+00,
        -3.29332415e+00, -3.62216436e+00,  3.92814684e-02,
        -4.75009527e+00, -1.02597828e+00,  3.06765160e-01,
        -4.75611619e+00],
       [-1.52461762e+00, -5.06263414e-01, -4.89835164e-01,
        -5.44598729e+00, -4.25420002e+00, -3.18466352e+00,
        -3.70165001e+00, -4.37637666e+00, -3.38096409e-01,
         9.18341025e-01],
       [ 1.18024366e-01, -3.11018024e+00,  2.75901422e+00,
        -3.81183859e+00,  4.41686111e-01, -3.27361812e+00,
        -2.14783101e+00,  1.14980027e-01, -1.94375906e+00,
         3.